# Load data

In [ ]:
import numpy as np
import pandas as pd
import gc

gc.enable()

# LOAD AND FREQUENCY-ENCODE
FE = ['EngineVersion','AppVersion','AvSigVersion','Census_OSVersion']
# LOAD AND ONE-HOT-ENCODE
OHE = [ 'RtpStateBitfield','IsSxsPassiveMode','DefaultBrowsersIdentifier',
        'AVProductStatesIdentifier','AVProductsInstalled', 'AVProductsEnabled',
        'CountryIdentifier', 'CityIdentifier', 
        'GeoNameIdentifier', 'LocaleEnglishNameIdentifier',
        'Processor', 'OsBuild', 'OsSuite',
        'SmartScreen','Census_MDC2FormFactor',
        'Census_OEMNameIdentifier', 
        'Census_ProcessorCoreCount',
        'Census_ProcessorModelIdentifier', 
        'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
        'Census_HasOpticalDiskDrive',
        'Census_TotalPhysicalRAM', 'Census_ChassisTypeName',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches',
        'Census_InternalPrimaryDisplayResolutionHorizontal',
        'Census_InternalPrimaryDisplayResolutionVertical',
        'Census_PowerPlatformRoleName', 'Census_InternalBatteryType',
        'Census_InternalBatteryNumberOfCharges',
        'Census_OSEdition', 'Census_OSInstallLanguageIdentifier',
        'Census_GenuineStateName','Census_ActivationChannel',
        'Census_FirmwareManufacturerIdentifier',
        'Census_IsTouchEnabled', 'Census_IsPenCapable',
        'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer',
        'Wdft_RegionIdentifier']

# LOAD ALL AS CATEGORIES
dtypes = {}
for x in FE+OHE: dtypes[x] = 'category'
dtypes['MachineIdentifier'] = 'str'
dtypes['HasDetections'] = 'int8'

# LOAD CSV FILE
train = pd.read_csv('../input/microsoft-malware-prediction/train.csv', usecols=dtypes.keys(), dtype=dtypes)
train.head()

# Downsample

In [ ]:
sm = 1000000
train = train.sample(sm)

gc.collect()
train.head()

# Frequency and One Hot Encoding

In [ ]:
import math

# CHECK FOR NAN
def nan_check(x):
    if isinstance(x,float):
        if math.isnan(x):
            return True
    return False

# FREQUENCY ENCODING
def encode_FE(df,col):
    d = df[col].value_counts(dropna=False)
    n = col+"_FE"
    df[n] = df[col].map(d)/d.max()
    return [n]

# ONE-HOT-ENCODE ALL CATEGORY VALUES THAT COMPRISE MORE THAN
# "FILTER" PERCENT OF TOTAL DATA AND HAS SIGNIFICANCE GREATER THAN "ZVALUE"
def encode_OHE(df, col, filter, zvalue, tar='HasDetections', m=0.5):
    cv = df[col].value_counts(dropna=False)
    cvd = cv.to_dict()
    vals = len(cv)
    th = filter * len(df)
    sd = zvalue * 0.5/ math.sqrt(th)
    #print(sd)
    n = []; ct = 0; d = {}
    for x in cv.index:
        try:
            if cv[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cv[x])
        except:
            if cvd[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cvd[x])
        if nan_check(x): r = df[df[col].isna()][tar].mean()
        else: r = df[df[col]==x][tar].mean()
        if abs(r-m)>sd:
            nm = col+'_BE_'+str(x)
            if nan_check(x): df[nm] = (df[col].isna()).astype('int8')
            else: df[nm] = (df[col]==x).astype('int8')
            n.append(nm)
            d[x] = 1
        ct += 1
        if (ct+1)>=vals: break
    return [n,d]

# ONE-HOT-ENCODING from dictionary
def encode_OHE_test(df,col,dt):
    n = []
    for x in dt: 
        n += encode_BE(df,col,x)
    return n

# BOOLEAN ENCODING
def encode_BE(df,col,val):
    n = col+"_BE_"+str(val)
    if nan_check(val):
        df[n] = df[col].isna()
    else:
        df[n] = df[col]==val
    df[n] = df[n].astype('int8')
    return [n]


In [ ]:
cols = []
dd = []

# ENCODE NEW
for x in FE:
    cols += encode_FE(train,x)
for x in OHE:
    tmp = encode_OHE(train,x,0.005,5)
    cols += tmp[0]
    dd.append(tmp[1])

# REMOVE OLD
for x in FE+OHE:
    del train[x]

gc.collect()
train.head()

# Split the training data into label and features

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train[cols], train['HasDetections'], test_size = 0.5, random_state=1)

# Helper functions

In [ ]:
from sklearn.metrics import accuracy_score

def train_and_get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict)
    return model, accuracy

In [ ]:
from sklearn import metrics

def get_AUC(model, X_test, y_test):
    y_predict_prob = model.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_prob[:,1], pos_label=1)
    AUC = metrics.auc(fpr, tpr)
    return AUC, fpr, tpr

In [ ]:
import matplotlib.pyplot as plt

def plot_ROC_curve(AUC, fpr, tpr):
    # The following line will tell Jupyter Notebook to keep the figures inside the explorer page 
    # rather than openng a new figure window:
    %matplotlib inline

    plt.figure()

    # Roc Curve:
    plt.plot(fpr, tpr, color='red', lw=2, 
             label='ROC Curve (area = %0.2f)' % AUC)

    # Random Guess line:
    plt.plot([0, 1], [0, 1], color='blue', lw=1, linestyle='--')

    # Defining The Range of X-Axis and Y-Axis:
    plt.xlim([-0.005, 1.005])
    plt.ylim([0.0, 1.01])

    # Labels, Title, Legend:
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

    plt.show()

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg, accuracy = train_and_get_score(logreg, X_train, X_test, y_train, y_test)

print(f"Accuracy score: {accuracy}")

AUC_logreg, fpr_logreg, tpr_logreg = get_AUC(logreg, X_test, y_test)

print(f"AUC: {AUC_logreg}")

plot_ROC_curve(AUC_logreg, fpr_logreg, tpr_logreg)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

randfor = RandomForestClassifier(n_estimators = 19, bootstrap = True, random_state=2)
randfor, accuracy = train_and_get_score(randfor, X_train, X_test, y_train, y_test)

print(f"Accuracy score: {accuracy}")

AUC_randfor, fpr_randfor, tpr_randfor = get_AUC(randfor, X_test, y_test)

print(f"AUC: {AUC_randfor}")

plot_ROC_curve(AUC_randfor, fpr_randfor, tpr_randfor)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dectree = DecisionTreeClassifier(random_state=3)
dectree, accuracy = train_and_get_score(dectree, X_train, X_test, y_train, y_test)

print(f"Accuracy score: {accuracy}")

AUC_dectree, fpr_dectree, tpr_dectree = get_AUC(dectree, X_test, y_test)

print(f"AUC: {AUC_dectree}")

plot_ROC_curve(AUC_dectree, fpr_dectree, tpr_dectree)

# KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=10)
#knn, accuracy = train_and_get_score(knn, X_train, X_test, y_train, y_test)

#print(f"Accuracy score: {accuracy}")

#AUC_knn, fpr_knn, tpr_knn = get_AUC(knn, X_test, y_test)

#print(f"AUC: {AUC_knn}")

#plot_ROC_curve(AUC_knn, fpr_knn, tpr_knn)

# ROC Curve comparisions

In [ ]:
%matplotlib inline

plt.figure()

# Roc Curve:
plt.plot(fpr_logreg, tpr_logreg, color='red', lw=2, 
            label='Logistic Regresssion ROC Curve (area = %0.2f)' % AUC_logreg)
plt.plot(fpr_randfor, tpr_randfor, color='green', lw=2, 
            label='Random Forest ROC Curve (area = %0.2f)' % AUC_randfor)
plt.plot(fpr_dectree, tpr_dectree, color='purple', lw=2, 
            label='Decision Tree ROC Curve (area = %0.2f)' % AUC_dectree)
#plt.plot(fpr_logreg, tpr_logreg, color='orange', lw=2, 
#            label='KNN ROC Curve (area = %0.2f)' % AUC_knn)

# Random Guess line:
plt.plot([0, 1], [0, 1], color='blue', lw=1, linestyle='--')

# Defining The Range of X-Axis and Y-Axis:
plt.xlim([-0.005, 1.005])
plt.ylim([0.0, 1.01])

# Labels, Title, Legend:
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

plt.show()